In [1]:
import os

import bs4
import dotenv
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import HuggingFaceEndpoint
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter

dotenv.load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["HUGGINGFACEHUB_API_TOKEN"] = os.getenv("HUGGINGFACEHUB_API_TOKEN")

# RAG against my machine

In [2]:
# Parameters
fp_data = "./data/dutch-news-articles.csv"

# llm_id = "Rijgersberg/GEITje-7B"
# llm_id = "GroNLP/bert-base-dutch-cased"
llm_id = "./ov_model_dir"

llm_pipeline_kwargs = {"max_new_tokens": 100}
llm_model_kwargs = {"ov_config": {"KV_CACHE_PRECISION": "u8",
                                  "DYNAMIC_QUANTIZATION_GROUP_SIZE": "32",
                                  "PERFORMANCE_HINT": "LATENCY",
                                  "NUM_STREAMS": "1",
                                  "CACHE_DIR": ""}}

chunk_size = 1_000
chunk_overlap = 200
N_docs = 10  # Set to None to process all documents

# embed_id = "sentence-transformers/all-mpnet-base-v2"
# embed_id = "sentence-transformers/all-MiniLM-L6-v2"  # Faster model
embed_id = "paraphrase-multilingual-MiniLM-L12-v2"  # Multilingual model, including Dutch

embed_model_kwargs = {"device": "cpu"}
embed_encode_kwargs = {"normalize_embeddings": False}

vectorstore_search_kwargs = {"k": 6}

question = "Welke kerncentrale is recent buiten bedrijf gesteld?"

In [3]:
llm = HuggingFacePipeline.from_model_id(model_id=llm_id,
                                        task="text-generation",
                                        backend="openvino",
                                        model_kwargs=llm_model_kwargs,
                                        pipeline_kwargs=llm_pipeline_kwargs)  # Add device_map="auto" once device= is removed
llm

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


INFO:nncf:NNCF initialized successfully. Supported frameworks detected: torch, tensorflow, onnx, openvino


C:\Users\denbe\AppData\Roaming\Python\Python39\site-packages\transformers\utils\import_utils.py:519: FutureWarning: `is_torch_tpu_available` is deprecated and will be removed in 4.41.0. Please use the `is_torch_xla_available` instead.
  warnings.warn(
Compiling the model to CPU ...


HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x0000020477EEB250>, model_id='./ov_model_dir', model_kwargs={'ov_config': {'KV_CACHE_PRECISION': 'u8', 'DYNAMIC_QUANTIZATION_GROUP_SIZE': '32', 'PERFORMANCE_HINT': 'LATENCY', 'NUM_STREAMS': '1', 'CACHE_DIR': ''}}, pipeline_kwargs={'max_new_tokens': 100})

In [4]:
loader = CSVLoader(file_path=fp_data, encoding="utf-8", source_column="url", metadata_columns=["datetime", "category", "url"])
docs = loader.load()

print(docs[0].page_content[:100])  # Example of the first 100 characters of the first document
print(docs[0].page_content[-100:])  # Example of the last 100 characters of the first document

title: Enige Litouwse kerncentrale dicht
content: De enige kerncentrale van Litouwen is oudjaarsavon
jvoorbeeld gas uit Rusland. De kerncentrale leverde bijna driekwart van de Litouwse energiebehoefte.


In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap, add_start_index=True)
splits = text_splitter.split_documents(docs[:N_docs] if N_docs else docs)

print(f"{len(splits)} splits in total")

21 splits in total


In [6]:
print(f"Metadata: {splits[0].metadata}")
print(f"Contents:\n\n{splits[0].page_content}")

Metadata: {'source': 'https://nos.nl/artikel/126231-enige-litouwse-kerncentrale-dicht.html', 'row': 0, 'datetime': '2010-01-01 00:49:00', 'category': 'Buitenland', 'url': 'https://nos.nl/artikel/126231-enige-litouwse-kerncentrale-dicht.html', 'start_index': 0}
Contents:

title: Enige Litouwse kerncentrale dicht
content: De enige kerncentrale van Litouwen is oudjaarsavond om 23.00 uur buiten gebruik gesteld. Dat verliep zonder problemen, aldus de directeur. Litouwen beloofde al in 2004 om de centrale te sluiten in ruil voor toetreding tot de Europese Unie. De EU wilde sluiting omdat de kerncentrale bij de stad Visiginas mogelijk niet veilig was. Nucleaire ramp De centrale is een grotere versie van die bij Tsjernobyl. Die ontplofte in 1986 en veroorzaakte een nucleaire wolk die over een groot deel van Europa trok. Dat was de grootste nucleaire ramp in de geschiedenis. Voor Litouwen betekent de sluiting dat het land een goedkope bron van energie kwijt is. Het wordt nu veel afhankelijker v

In [7]:
embed_model = HuggingFaceEmbeddings(model_name=embed_id,
                                    model_kwargs=embed_model_kwargs,
                                    encode_kwargs=embed_encode_kwargs)

In [8]:
vectorstore = Chroma.from_documents(documents=splits, embedding=embed_model)  # Investigate persist_directory to store the vectorstore

In [9]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs=vectorstore_search_kwargs)
retrieved_docs = retriever.invoke(question)

print(f"{len(retrieved_docs)} retrieved docs")

print("Contents:")
for i, doc in enumerate(retrieved_docs):
    print(f"Doc {i}: {doc.page_content[:200]}")

6 retrieved docs
Contents:
Doc 0: title: Enige Litouwse kerncentrale dicht
content: De enige kerncentrale van Litouwen is oudjaarsavond om 23.00 uur buiten gebruik gesteld. Dat verliep zonder problemen, aldus de directeur. Litouwen be
Doc 1: title: Enige Litouwse kerncentrale dicht
content: De enige kerncentrale van Litouwen is oudjaarsavond om 23.00 uur buiten gebruik gesteld. Dat verliep zonder problemen, aldus de directeur. Litouwen be
Doc 2: content: Vannacht is een recordaantal sms'jes verstuurd. De drie grootste telecomaanbieders hadden het daar erg druk mee. KPN verwerkte er 27,7 miljoen, ruim drie miljoen meer dan vorig jaar. Er werd 
Doc 3: content: Vannacht is een recordaantal sms'jes verstuurd. De drie grootste telecomaanbieders hadden het daar erg druk mee. KPN verwerkte er 27,7 miljoen, ruim drie miljoen meer dan vorig jaar. Er werd 
Doc 4: title: Obama krijgt rapporten over aanslag
content: President Obama heeft de eerste rapporten gekregen over de mislukte aanslag op een

In [10]:
# prompt = hub.pull("rlm/rag-prompt")
prompt = ChatPromptTemplate.from_messages(
    [("human", "Je bent een assistent voor het beantwoorden van vragen. Gebruik de volgende stukjes opgehaalde context om de vraag te beantwoorden. Als je het antwoord niet weet, zeg dan gewoon dat je het niet weet. Gebruik maximaal drie zinnen en houd het antwoord beknopt.\nVraag: {question} \nContext: {context} \nAntwoord:")])

example_messages = prompt.invoke({"context": "filler context", "question": "filler question"}).to_messages()
print(example_messages)

[HumanMessage(content='Je bent een assistent voor het beantwoorden van vragen. Gebruik de volgende stukjes opgehaalde context om de vraag te beantwoorden. Als je het antwoord niet weet, zeg dan gewoon dat je het niet weet. Gebruik maximaal drie zinnen en houd het antwoord beknopt.\nVraag: filler question \nContext: filler context \nAntwoord:')]


In [11]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [12]:
rag_chain = ({"context": retriever | format_docs, "question": RunnablePassthrough()}
             | prompt
             | llm
             | StrOutputParser())

In [13]:
answer = rag_chain.invoke(question)

print(answer)

Human: Je bent een assistent voor het beantwoorden van vragen. Gebruik de volgende stukjes opgehaalde context om de vraag te beantwoorden. Als je het antwoord niet weet, zeg dan gewoon dat je het niet weet. Gebruik maximaal drie zinnen en houd het antwoord beknopt.
Vraag: Welke kerncentrale is recent buiten bedrijf gesteld? 
Context: title: Enige Litouwse kerncentrale dicht
content: De enige kerncentrale van Litouwen is oudjaarsavond om 23.00 uur buiten gebruik gesteld. Dat verliep zonder problemen, aldus de directeur. Litouwen beloofde al in 2004 om de centrale te sluiten in ruil voor toetreding tot de Europese Unie. De EU wilde sluiting omdat de kerncentrale bij de stad Visiginas mogelijk niet veilig was. Nucleaire ramp De centrale is een grotere versie van die bij Tsjernobyl. Die ontplofte in 1986 en veroorzaakte een nucleaire wolk die over een groot deel van Europa trok. Dat was de grootste nucleaire ramp in de geschiedenis. Voor Litouwen betekent de sluiting dat het land een goedk